In [1]:
from IPython.display import display
import gc
import joblib as jb
from tqdm.auto import tqdm, trange
import os

# data manipulation
import numpy as np
import pandas as pd
import sqlite3 as sq
from sklearn import preprocessing, pipeline

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
con = sq.connect("switrs.sqlite")
cur = con.cursor()

In [3]:
tables = [name[0] for name in cur.execute("SELECT name FROM sqlite_master")]
tables


['case_ids', 'collisions', 'victims', 'parties']

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
collision_head_2020 = pd.read_sql("""SELECT *
            FROM collisions WHERE collision_date BETWEEN '2020-01-01'
            AND '2020-12-31';""", con)


In [6]:
collision_head_2020.shape

(366417, 75)

In [7]:
columns = list(collision_head_2020.columns)

In [8]:
columns

['case_id',
 'jurisdiction',
 'officer_id',
 'reporting_district',
 'chp_shift',
 'population',
 'county_city_location',
 'county_location',
 'special_condition',
 'beat_type',
 'chp_beat_type',
 'city_division_lapd',
 'chp_beat_class',
 'beat_number',
 'primary_road',
 'secondary_road',
 'distance',
 'direction',
 'intersection',
 'weather_1',
 'weather_2',
 'state_highway_indicator',
 'caltrans_county',
 'caltrans_district',
 'state_route',
 'route_suffix',
 'postmile_prefix',
 'postmile',
 'location_type',
 'ramp_intersection',
 'side_of_highway',
 'tow_away',
 'collision_severity',
 'killed_victims',
 'injured_victims',
 'party_count',
 'primary_collision_factor',
 'pcf_violation_code',
 'pcf_violation_category',
 'pcf_violation',
 'pcf_violation_subsection',
 'hit_and_run',
 'type_of_collision',
 'motor_vehicle_involved_with',
 'pedestrian_action',
 'road_surface',
 'road_condition_1',
 'road_condition_2',
 'lighting',
 'control_device',
 'chp_road_type',
 'pedestrian_collision',


In [9]:
drop_column_list= [
 'jurisdiction',
 'officer_id',
 'reporting_district',
 'chp_shift',
 'county_city_location',
 'special_condition',
 'beat_type',
 'chp_beat_type',
 'city_division_lapd',
 'chp_beat_class',
 'beat_number',
 'distance',
 'direction',
 'intersection',
 'state_highway_indicator',
 'caltrans_county',
 'caltrans_district',
 'state_route',
 'route_suffix',
 'postmile_prefix',
 'postmile',
 'ramp_intersection',
 'control_device',
 'chp_road_type',
 'not_private_property',
 'alcohol_involved',
 'statewide_vehicle_type_at_fault',
 'chp_vehicle_type_at_fault',
 'severe_injury_count',
 'other_visible_injury_count',
 'complaint_of_pain_injury_count',
 'pedestrian_killed_count',
 'pedestrian_injured_count',
 'bicyclist_killed_count',
 'bicyclist_injured_count',
 'motorcyclist_killed_count',
 'motorcyclist_injured_count',
 'primary_ramp',
 'secondary_ramp',
 'latitude',
 'longitude',
 'collision_date',
 'collision_time']

In [10]:
collision_head_2020.columns

Index(['case_id', 'jurisdiction', 'officer_id', 'reporting_district',
       'chp_shift', 'population', 'county_city_location', 'county_location',
       'special_condition', 'beat_type', 'chp_beat_type', 'city_division_lapd',
       'chp_beat_class', 'beat_number', 'primary_road', 'secondary_road',
       'distance', 'direction', 'intersection', 'weather_1', 'weather_2',
       'state_highway_indicator', 'caltrans_county', 'caltrans_district',
       'state_route', 'route_suffix', 'postmile_prefix', 'postmile',
       'location_type', 'ramp_intersection', 'side_of_highway', 'tow_away',
       'collision_severity', 'killed_victims', 'injured_victims',
       'party_count', 'primary_collision_factor', 'pcf_violation_code',
       'pcf_violation_category', 'pcf_violation', 'pcf_violation_subsection',
       'hit_and_run', 'type_of_collision', 'motor_vehicle_involved_with',
       'pedestrian_action', 'road_surface', 'road_condition_1',
       'road_condition_2', 'lighting', 'control_devi

In [13]:
collision_2020 = collision_head_2020.drop(drop_column_list, axis=1, inplace = False)

In [14]:
type(collision_2020)

pandas.core.frame.DataFrame

In [ ]:
case_ids_2020 = pd.read_sql("""SELECT *
            FROM case_ids;""", con)


In [24]:
case_ids_2020

2020

In [22]:
distinct_years = pd.read_sql("""SELECT DISTINCT 
         strftime('%Y', collision_date) from collisions ORDER BY collision_date;""", con)

In [23]:
distinct_years

,"strftime('%Y', collision_date)"
0,2001
1,2002
2,2003
3,2004
4,2005
5,2006
6,2007
7,2008
8,2009
9,2010


In [15]:
collision_head_10.collision_date

0    2020-03-14
1    2020-07-26
2    2020-10-05
3    2020-08-06
4    2020-10-05
5    2020-08-06
6    2020-10-05
7    2020-08-07
8    2020-10-02
9    2020-08-13
Name: collision_date, dtype: object

In [11]:
collision_head_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 75 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   case_id                          10 non-null     object 
 1   jurisdiction                     10 non-null     int64  
 2   officer_id                       10 non-null     object 
 3   reporting_district               3 non-null      object 
 4   chp_shift                        10 non-null     object 
 5   population                       10 non-null     object 
 6   county_city_location             10 non-null     object 
 7   county_location                  10 non-null     object 
 8   special_condition                10 non-null     object 
 9   beat_type                        10 non-null     object 
 10  chp_beat_type                    10 non-null     object 
 11  city_division_lapd               0 non-null      object 
 12  chp_beat_class           

Tables' features:

